<a href="https://colab.research.google.com/github/CH22B093/Task/blob/main/Breakout_AI_Task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importing Libraries


In [ ]:
import yfinance as yf
import pandas as pd

# Function to get Option chain data

In [ ]:
import yfinance as yf
import pandas as pd

def get_option_chain_data(instrument_name: str, expiry_date: str, side: str) -> pd.DataFrame:
    try:
        ticker = instrument_name + ".NS"
        yf_ticker = yf.Ticker(ticker)
        option_chain = yf_ticker.option_chain(expiry_date)

        options_data = {}

        if side == "CE":
            for call in option_chain.calls.itertuples():
                strike = call.strike
                if strike not in options_data:
                    options_data[strike] = {'instrument': instrument_name, 'strike': strike, 'expiryDate': expiry_date, 'side': 'CE', 'bid/ask': 0}
                options_data[strike]['bid/ask'] = max(options_data[strike]['bid/ask'], call.ask)
        elif side == "PE":
            for put in option_chain.puts.itertuples():
                strike = put.strike
                if strike not in options_data:
                    options_data[strike] = {'instrument': instrument_name, 'strike': strike, 'expiryDate': expiry_date, 'side': 'PE', 'bid/ask': 0}
                options_data[strike]['bid/ask'] = max(options_data[strike]['bid/ask'], put.bid)
        else:
            raise ValueError("Invalid side value. Use 'CE' for Call or 'PE' for Put.")

        df = pd.DataFrame(options_data.values())
        return df

    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Function to calculate margin and premium

In [ ]:
def calculate_margin_and_premium(option_data):
  lot_size = 50
  for index, row in option_data.iterrows():
    margin = 10000
    if row['side'] == 'call':
      premium = row['ask'] * lot_size
    elif row['side'] == 'put':
      premium = row['bid'] * lot_size
    else:
      premium = 0

    option_data.loc[index, 'margin'] = margin
    option_data.loc[index, 'premium'] = premium

  return option_data